In [12]:
import pandas as pd
from datetime import timedelta, date
import pickle
import matplotlib as plt

pd.set_option('display.max_columns',13)

In [13]:
"""
start = date(2018, 3, 1)
end = date(2018, 6, 1)
"""

'\nstart = date(2018, 3, 1)\nend = date(2018, 6, 1)\n'

In [14]:
"""
df= pd.DataFrame()
for sdate in range((end - start).days):
    fdate = (start + timedelta(sdate)).strftime('%y%m%d')
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_" + fdate + ".txt"
        #print(url)
    try:
        data = pd.read_csv(url)
        data['file'] = fdate
        df = df.append(data)
    except:
        print(fdate + ' didnt work')
"""

'"\ndf= pd.DataFrame()\nfor sdate in range((end - start).days):\n    fdate = (start + timedelta(sdate)).strftime(\'%y%m%d\')\n    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_" + fdate + ".txt"\n        #print(url)\n    try:\n        data = pd.read_csv(url)\n        data[\'file\'] = fdate\n        df = df.append(data)\n    except:\n        print(fdate + \' didnt work\')\n'

In [15]:
##pickle.dump( df, open("turns.p", "wb"))

In [16]:
df = pickle.load( open("turns.p", "rb"))

In [34]:
df.rename(columns = {'EXITS                                                               ':'EXITS'}, inplace = True)
df["DATETIME"] = pd.to_datetime(df["DATE"] +" "+df['TIME'], format ='%m/%d/%Y %H:%M:%S')
df["turnstileID"] = df["C/A"] +" "+ df["UNIT"] + " "+ df["SCP"]
df = df[df.TIME.str.contains(':00:00', regex=False)]


In [35]:
stations_list = []
for station in set(df.STATION):
    single= df[df['STATION'] == station]
    single= single.sort_values(by =['turnstileID','DATETIME'])
    ##single = single.sort_values(by="DATETIME")
    ##single = single.groupby(by= "DATETIME").sum()
    single['STATION'] = station
    ##single['DATETIME'] = single.index
    single['WEEKDAY'] = single['DATETIME'].dt.day_name()
    single['entry_diff'] = single.ENTRIES.shift(-1)-single.ENTRIES
    single['exit_diff'] = single.EXITS.shift(-1)-single.EXITS
    single['time_diff'] = single.DATETIME.shift(-1)-single.DATETIME

    stations_list.append(single)
all_df= pd.concat(stations_list)

In [19]:
all_df.describe()

,ENTRIES,EXITS,entry_diff,exit_diff,time_diff
count,2.564404e+06,2.564404e+06,2.564028e+06,2.564028e+06,2564028
mean,3.912342e+07,3.224968e+07,2.081979e+03,1.109915e+03,0 days 00:18:52.425120
std,2.045653e+08,1.898767e+08,1.233716e+07,1.094915e+07,3 days 17:34:30.066557
min,0.000000e+00,0.000000e+00,-2.117603e+09,-2.145033e+09,-91 days +00:20:55
25%,5.076080e+05,2.492142e+05,9.000000e+00,8.000000e+00,0 days 04:00:00
50%,2.582216e+06,1.456342e+06,7.400000e+01,5.100000e+01,0 days 04:00:00
75%,6.884556e+06,4.824353e+06,2.430000e+02,1.650000e+02,0 days 04:00:00
max,2.130068e+09,2.145850e+09,2.122292e+09,2.145033e+09,42 days 00:00:00


In [36]:
single.describe()

,ENTRIES,EXITS,entry_diff,exit_diff,time_diff
count,4.913000e+03,4.913000e+03,4.912000e+03,4.912000e+03,4912
mean,4.902134e+06,2.480073e+06,-2.534237e+03,1.703080e+03,0 days 00:26:38.452768
std,6.469847e+06,2.681138e+06,1.801335e+05,9.235841e+04,3 days 16:07:00.093838
min,9.714000e+03,1.752050e+05,-1.137952e+07,-7.779260e+05,-91 days +03:00:00
25%,2.300350e+05,4.691580e+05,1.000000e+00,1.400000e+01,0 days 04:00:00
50%,5.417950e+05,1.990403e+06,1.300000e+01,3.400000e+01,0 days 04:00:00
75%,1.229097e+07,2.729817e+06,1.040000e+02,9.100000e+01,0 days 04:00:00
max,1.651094e+07,9.469258e+06,3.120056e+06,5.777192e+06,0 days 08:00:00


,STATION,DATETIME,ENTRIES,EXITS,entry_diff,exit_diff
8817,138/GRAND CONC,2018-05-25 20:00:00,4093629.0,3617843.0,4.938498e+08,477117676.5
2216,104 ST,2018-05-25 20:00:00,563566558.5,313470043.0,4.517711e+08,-309126903.0
146703,EASTCHSTER/DYRE,2018-05-25 21:00:00,6251261.0,8032166.0,2.705345e+08,270618436.5
108500,BRONX PARK EAST,2018-05-25 21:00:00,3854914.0,1708787.0,9.894389e+07,-1078538.5
241440,MONTROSE AV,2018-05-25 20:00:00,6421536.0,1220615.0,6.499364e+07,-3437816.5
115792,CENTRAL PK N110,2018-05-25 20:00:00,31119761.0,11981715.5,7.913624e+06,2717004.0
188397,HALSEY ST,2018-05-25 21:00:00,2461784.0,1239265.0,7.504910e+06,2146707.5
120595,CHURCH AV,2018-05-25 21:00:00,18192100.5,1151455.5,6.616324e+06,620323.0
4971,116 ST-COLUMBIA,2018-05-25 21:00:00,12391776.5,5548652.5,6.440028e+06,-1949296.0
22296,170 ST,2018-05-25 20:00:00,4608062.0,3543408.5,5.792893e+06,4592671.0


,STATION,TIME,ENTRIES,EXITS,entry_diff,exit_diff
0,1 AV,00:00:00,225147474065,120637555927,5.955700e+04,44299.0
1,1 AV,03:00:00,45532888268,24349461536,-6.002100e+04,-131455.0
2,1 AV,04:00:00,230769139859,123444457639,9.462800e+04,202991.0
3,1 AV,07:00:00,45532828247,24349330081,6.025500e+04,78479.0
4,1 AV,07:45:55,2585498693,1182790841,8.700000e+02,3272.0
5,1 AV,08:00:00,228183336723,122261284067,3.173410e+05,375441.0
6,1 AV,11:00:00,45532889372,24349411832,5.590500e+04,60108.0
7,1 AV,12:00:00,230769551828,123445036071,3.556590e+05,336712.0
8,1 AV,15:00:00,45532945277,24349471940,9.596700e+04,90275.0
9,1 AV,16:00:00,230769907487,123445372783,4.709430e+05,496347.0
